In [1]:
import os
import json
import csv

import pandas as pd

## Json to CSV files/joining

In [9]:
data_folder_videos='/media/aruiz/data/videos_data/'
filename1='general_videos_data2.json'
filename2='general_videos_data.csv'
file1=os.path.join(data_folder_videos,filename1)
file2=os.path.join(data_folder_videos,filename2)
file3= "json_errors.txt"

fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']
output=open(file2, 'a')

writer = csv.DictWriter(output, fieldnames=fieldnames)
# writer.writeheader()
i=0

with open (file1,'r') as f:
    for line in f:
        i+=1
        if i>0:
            try:
                clean_dict={}
                json_dict=json.loads(line)
                videos= json_dict.get("items",[])
                if (videos!=[]):
                    clean_dict['id']= videos[0]['id']
                    clean_dict['publishedAt']= videos[0]['snippet']['publishedAt']
                    clean_dict['channelId']= videos[0]['snippet']['channelId']
                    clean_dict['title']= videos[0]['snippet']['title']
                    
                    if 'channelTitle' in videos[0]['snippet']:
                        clean_dict['channelTitle']= videos[0]['snippet']['channelTitle']
                    
                    if 'viewCount' in videos[0]['statistics']:
                        clean_dict['viewCount']=  videos[0]['statistics']['viewCount']

                    if 'description' in videos[0]['snippet']:
                        clean_dict['description']= videos[0]['snippet']['description']

                    if 'tags' in videos[0]['snippet']:
                        clean_dict['tags']= videos[0]['snippet']['tags']

                    if 'categoryId' in videos[0]['snippet']:
                        clean_dict['categoryId']= videos[0]['snippet']['categoryId']

                    if 'liveBroadcastContent' in videos[0]['snippet']:
                        clean_dict['liveBroadcast']= videos[0]['snippet']['liveBroadcastContent']

                    if 'favoriteCount' in videos[0]['statistics']:
                        clean_dict['favoriteCount']= videos[0]['statistics']['favoriteCount']

                    if 'commentCount' in videos[0]['statistics']:
                        clean_dict['commentCount']= videos[0]['statistics']['commentCount']

                    if 'duration' in videos[0]['contentDetails']:
                        clean_dict['duration']= videos[0]['contentDetails']['duration']

                    if 'definition' in videos[0]['contentDetails']:
                        clean_dict['definition']= videos[0]['contentDetails']['definition']

                    if 'caption' in videos[0]['contentDetails']:
                        clean_dict['caption']= videos[0]['contentDetails']['caption']

                    if 'topicDetails' in videos[0]:
                        if 'topicIds' in videos[0]['topicDetails']:
                            clean_dict['topicIds']= videos[0]['topicDetails']['topicIds']
                        if 'relevantTopicIds' in videos[0]['topicDetails']:
                            clean_dict['freebaseTopIds']= videos[0]['topicDetails']['relevantTopicIds']

                    if 'likeCount' in videos[0]['statistics']:
                        clean_dict['likeCount']= videos[0]['statistics']['likeCount']

                    if 'dislikeCount' in videos[0]['statistics']:
                        clean_dict['dislikeCount']= videos[0]['statistics']['dislikeCount']

                writer.writerow(clean_dict)
                
            except json.JSONDecodeError:
                errors=open(file3, 'a')
                errors.write(str(i)+'\n')
                errors.close()
                continue

In [2]:
#total=9632501
#total 2 = 4365722
#total general = 13998223
#no duplicates = 11752812

### Counting lines in csv

In [29]:
with open (file4, 'r') as csvfile:
    fileObj=csv.reader(csvfile)
    num = sum(1 for row in fileObj)
num

11752812

## Clean duplicates

In [28]:
import os
import csv
import pandas as pd

total_g = 13998222
fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']
data_folder_videos='/media/aruiz/data/videos_data/'
filename2='general_videos_data.csv'
file4=os.path.join(data_folder_videos,'general_videos_clean.csv')
file2=os.path.join(data_folder_videos,filename2)
# df=dd.read_csv(file2,sep =',', header=0,encoding='utf-16')
# df.head()

# --------------- divide csv in chunks ---------------------------------------------------
df = pd.read_csv(file2, header=0, names=fieldnames, encoding='utf-8', chunksize=1000000)
ids=set()
j=0

for chunk in df:
    j+=1
    chunk = chunk.drop_duplicates(['id'], keep='first')
    chunk = chunk[~chunk['id'].isin(ids)]
    ids.update(chunk['id'].values)
    chunk.to_csv(file4, encoding='utf-8', index=False, chunksize=1000000, mode='a')

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Save each community videos in a csv

In [27]:
data_folder_channels='/media/aruiz/data/videos_data/'
file1=os.path.join(data_folder_channels,'general_videos_clean.csv')
file2=os.path.join(data_folder_channels,'node_data_bc.csv')
fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']

#community chosen
community_num=[9,22,85,76,71,37,87]

for ind in range (1,len(community_num)):

    # create csv to save community videos
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')

    # Data frame node data from graph communities
    df1=pd.read_csv(file2)
    channel_com = df1.drop(df1[df1['modularity_class']!= community_num[ind]].index)
    com_ids=set()
    com_ids.update(channel_com['Id'].values)

    # Data frame all videos 
    df2 = pd.read_csv(file1, header=0, names=fieldnames, encoding='utf-8', chunksize=1000000)

    for chunk in df2:
        chunk = chunk[chunk['channelId'].isin(com_ids)]
        chunk.to_csv(file3, encoding='utf-8', index=False, chunksize=1000000, mode='a')

## Join csv comments

In [49]:
community_num=[9,22,85,76,71,37,87]
data_comments='/media/aruiz/data/comments_data/'
data_islp='/media/aruiz/data/videos_data/islp/'
f2=os.path.join(data_comments,'comments.csv')
fieldnames=['channelId','category','text']
out1=open(f2, 'a')
writer = csv.DictWriter(out1, fieldnames=fieldnames)
writer.writeheader()

# for ind in range (1, len(community_num)):
for ind in range (0,len(community_num)):
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
    fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
    df=pd.read_csv(fileName1)
    ids=df.videoId
    df2=pd.read_csv(f,index_col='id')
    
    for i in range(0, len(df)):
#         temp=df2.loc[df2['id'] == df.videoId[i]]
#         df3=df2[df2['id']==df.videoId[i]]
        dictio={}
        dictio['channelId']= df2.at[df.videoId[i], 'channelId']
        dictio['category']=community_num[ind]
        dictio['text']=df.textOriginal[i]
        
        writer.writerow(dictio)

out1.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Join csv comment and video

In [6]:
community_num=[9,22,85,76,71,37,87]
data_comments='/media/aruiz/data/comments_data/'
data_islp='/media/aruiz/data/videos_data/islp/'
f2=os.path.join(data_comments,'comments.csv')
fieldnames=['channelId','category','text']
out1=open(f2, 'a')
writer = csv.DictWriter(out1, fieldnames=fieldnames)
writer.writeheader()

# for ind in range (1, len(community_num)):
for ind in range (0,len(community_num)):
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
    fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
    df=pd.read_csv(fileName1)
    ids=df.videoId
    df2=pd.read_csv(f,index_col='id')
    print(df.head())
    break
    for i in range(0, len(df)):
#         temp=df2.loc[df2['id'] == df.videoId[i]]
#         df3=df2[df2['id']==df.videoId[i]]
        dictio={}
        dictio['channelId']= df2.at[df.videoId[i], 'channelId']
        dictio['category']=community_num[ind]
        dictio['text']=df.textOriginal[i]
        
        writer.writerow(dictio)

out1.close()

       videoId  year                   commentId      authorName  \
0  UDFJRgCaK7Q  2008  Ugz6TPBQBEmgntHPyVF4AaABAg  The 1ndiegamer   
1  UDFJRgCaK7Q  2008  Ugyh5pnsIXLvMapckVt4AaABAg  The 1ndiegamer   
2  DIJKuJY4tnY  2008  Ugz6da2h99SBN-GHgi94AaABAg        HaldorZX   
3  DIJKuJY4tnY  2008  UgyPrC6kg7RNpu51EZB4AaABAg        HaldorZX   
4  DIJKuJY4tnY  2008  Ugxu743RtFdY9SH7D-14AaABAg       lothrazar   

            authorChannelId  \
0  UCoGQWna5-JX3CjqHMNGRNNA   
1  UCoGQWna5-JX3CjqHMNGRNNA   
2  UCSEm_uryLozckEgRenP-dIA   
3  UCSEm_uryLozckEgRenP-dIA   
4  UC5tp0GG3-cvlnflj5NuA9xQ   

                                         textDisplay  \
0  i think it&#39;s the tree stump, could be a a ...   
1  Lothrazar. use the dream rod on the tree stump...   
2  whoops sorry posted that half way through the ...   
3  lol kinda pointless to say so now but by the l...   
4  Thanks, and it wasn&#39;t a spoiler, I usualy ...   

                                        textOriginal  likeCount  pY

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
len(df)

50890

## Join comments including all data

In [6]:
community_num=[9,22,85,76,71,37,87]
data_comments='/media/aruiz/data/comments_data/'
data_islp='/media/aruiz/data/videos_data/islp/'
f2=os.path.join('data/comments3.csv')
fieldnames=['id','commentYear','commentMonth','commentDay','likeCount','channelId','videoTags','channelTitle','videoId',
            'videoYear','videoMonth','videoViews','category','commentText']
out1=open(f2, 'a')
writer = csv.DictWriter(out1, fieldnames=fieldnames)
writer.writeheader()

# for ind in range (1, len(community_num)):
for ind in range (0,len(community_num)):
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
    fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
    df=pd.read_csv(fileName1)
    ids=df.videoId
    df2=pd.read_csv(f,index_col='id')
    for i in range(0, len(df)):
        dictio={}
        dictio['id']= df.commentId[i]
        dictio['commentYear']=  df.pYear[i]
        dictio['commentMonth']= df.pMonth[i]
        dictio['commentDay']= df.pDay[i]
        dictio['likeCount']= df.likeCount[i]
        dictio['channelId']= df2.at[df.videoId[i], 'channelId']
        dictio['videoId']= df.videoId[i]
        dictio['videoTags']= df2.at[df.videoId[i], 'tags']
        dictio['channelTitle']= df2.at[df.videoId[i], 'channelTitle']
        dictio['videoYear']= df2.at[df.videoId[i], 'year']
        dictio['videoMonth']= df2.at[df.videoId[i], 'month']
        dictio['videoViews']= df2.at[df.videoId[i], 'viewCount']
        dictio['category']=community_num[ind]
        dictio['commentText']=df.textOriginal[i]
        
        writer.writerow(dictio)

out1.close()

FileNotFoundError: File b'/media/aruiz/data/comments_data/community9_allPositive.csv' does not exist

In [9]:
df.columns

Index(['videoId', 'year', 'commentId', 'authorName', 'authorChannelId',
       'textDisplay', 'textOriginal', 'likeCount', 'pYear', 'pMonth', 'pDay'],
      dtype='object')

In [14]:
videos.shape

(163198, 22)

In [16]:
fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
comments=pd.read_csv(fileName1)

In [17]:
comments.columns

Index(['videoId', 'year', 'commentId', 'authorName', 'authorChannelId',
       'textDisplay', 'textOriginal', 'likeCount', 'pYear', 'pMonth', 'pDay'],
      dtype='object')

In [29]:
data_comments='data/'
f1=os.path.join(data_comments,'comments2.csv')
f2=os.path.join(data_comments,'comments_lan.csv')
comments1=pd.read_csv(f1)
# comments2=pd.read_csv(f2,index_col='text')
comments2=pd.read_csv(f2)
comments1['language']=comments2.language.apply(lambda x: x)
    
#     comments3=comments2[comments1.videoId]
#     try:
#         comments1['lan'][i]=comments2.at[comments1.commentText[i], 'language']
#     except KeyError:
#         print(comments1.commentText[i])
#         print(comments2)
    

In [30]:
comments1.columns

Index(['id', 'commentYear', 'commentMonth', 'commentDay', 'likeCount',
       'channelId', 'videoTags', 'channelTitle', 'videoYear', 'videoMonth',
       'videoViews', 'category', 'commentText', 'language'],
      dtype='object')

In [20]:
a=comments1.videoId.value_counts().values==1

In [28]:
j=0
b=[]
for i in range (0, len(a)):
    if a[i]==True:
        j+=1
        b.append(i)

In [31]:
# comments1.to_csv('data/comments_lan3.csv', index=False)

In [37]:
df3 = pd.read_csv('data/comments_translated2.csv')
df4 = pd.read_csv('data/comments_lan3.csv')
# df5=df4[df4['language']!='und']

In [40]:
print(df3.columns)
df4['text']=df4['commentText']


Index(['channelId', 'community', 'text', 'language', 'translation'], dtype='object')


In [41]:
print(df4.columns)

Index(['id', 'commentYear', 'commentMonth', 'commentDay', 'likeCount',
       'channelId', 'videoTags', 'channelTitle', 'videoYear', 'videoMonth',
       'videoViews', 'category', 'commentText', 'language', 'text'],
      dtype='object')


In [26]:
df4=df3[df3['language']!='und']

In [42]:
# df4.to_csv('data/comments_cleaned_all.csv', index=False)

In [43]:
abs_paths=['data/comments_translated2.csv','data/comments_cleaned_all.csv']
dfs = pd.concat([pd.read_csv(abs_path, usecols=['text'])for abs_path in abs_paths])

In [63]:
result=result.drop_duplicates(keep='first')

In [71]:
result.shape

(500268, 16)

In [73]:
b=b[b['text']!='']

In [74]:
b.shape

(316043, 15)

In [103]:
a=pd.read_csv('data/comments_translated2.csv')
b=pd.read_csv('data/comments_cleaned_all.csv')
result=b.merge(a, on=['text','channelId'])

In [104]:
result.shape

(309244, 18)

In [105]:
result.to_csv('data/comments_all_merged.csv', index=False)

In [112]:
data=pd.read_csv('data/comments_all_merged.csv',index_col='id')

In [113]:
data=data.drop_duplicates(keep='first')

In [115]:
data.shape

(270868, 17)

In [ ]:
data=pd.read_csv('data/comments_all_merged.csv',index_col='id')

In [117]:
data

,commentYear,commentMonth,commentDay,likeCount,channelId,videoTags,channelTitle,videoYear,videoMonth,videoViews,category,commentText,language_x,text,community,language_y,translation
id,,,,,,,,,,,,,,,,,
Ugz6TPBQBEmgntHPyVF4AaABAg,2008,11,28,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,245.0,9,"i think it's the tree stump, could be a a slee...",en,"i think it's the tree stump, could be a a slee...",9,en,"i think it's the tree stump, could be a a slee..."
Ugyh5pnsIXLvMapckVt4AaABAg,2008,11,28,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,245.0,9,Lothrazar. use the dream rod on the tree stump...,en,Lothrazar. use the dream rod on the tree stump...,9,en,Lothrazar. use the dream rod on the tree stump...
Ugz6da2h99SBN-GHgi94AaABAg,2009,3,21,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,239.0,9,whoops sorry posted that half way through the ...,en,whoops sorry posted that half way through the ...,9,en,whoops sorry posted that half way through the ...
UgyPrC6kg7RNpu51EZB4AaABAg,2009,3,21,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,239.0,9,lol kinda pointless to say so now but by the l...,en,lol kinda pointless to say so now but by the l...,9,en,lol kinda pointless to say so now but by the l...
Ugxu743RtFdY9SH7D-14AaABAg,2009,1,15,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,239.0,9,"Thanks, and it wasn't a spoiler, I usualy have...",en,"Thanks, and it wasn't a spoiler, I usualy have...",9,en,"Thanks, and it wasn't a spoiler, I usualy have..."
UgzR7burTjjX_MKriWh4AaABAg,2008,11,26,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,239.0,9,"Regarding the sleeping one you didn't find, if...",en,"Regarding the sleeping one you didn't find, if...",9,en,"Regarding the sleeping one you didn't find, if..."
UgwdKArf9tHHiqclBV14AaABAg,2008,11,23,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,256.0,9,You can do that in this game too. Lothrazar di...,en,You can do that in this game too. Lothrazar di...,9,en,You can do that in this game too. Lothrazar di...
UgzXhb_7FVMYi1y5Gcx4AaABAg,2008,11,23,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,256.0,9,You seem to have forgotten that this is Soul B...,en,You seem to have forgotten that this is Soul B...,9,en,You seem to have forgotten that this is Soul B...
UgxsLZjFCy40CGQK7IF4AaABAg,2008,11,22,0,UC5tp0GG3-cvlnflj5NuA9xQ,"['LP', 'blind', 'Lets', 'play', 'SNES', 'enix'...",lothrazar,2008.0,11.0,256.0,9,You seem to have completely forgotten your abi...,en,You seem to have completely forgotten your abi...,9,en,You seem to have completely forgotten your abi...


In [118]:
data.translation.value_counts()

cool                                                                                                                                                                                         205
lol                                                                                                                                                                                          149
Cool                                                                                                                                                                                         132
Lol                                                                                                                                                                                          116
Hi                                                                                                                                                                                            90
I                                  